# NBA Data Analysis for Prediction Model

## Table of Contents <a id="table_of_contents"></a>

0. [Download & Import Libraries](#imports)
1. [Import Data](#import_data)
2. Data Exploration
3. Data Cleaning
4. Data Wrangling
5. Data Visualization

## 0. Downloading and Importing Libraries <a id="imports"></a>

[Back to top](#table_of_contents)

In [ ]:
!pip install opendatasets
pip install pypyodbc
pip install geopy

In [ ]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib
from matplotlib import pyplot as plt
import opendatasets as od
import os
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder

## 1. Import Data <a id="import_data"></a>

[Back to top](#table_of_contents)

In [ ]:
#create connection to database file in Kaggle
dataset = "https://www.kaggle.com/datasets/wyattowalsh/basketball"
od.download(dataset)

In [ ]:
# 1.Connect to SQL database
connection = sql.connect('basketball/nba.sqlite')
# Test SQL connectivity to database by writing a SQL query
test_query = "SELECT * FROM game"
# Load the data into a pandas DataFrame
df = pd.read_sql_query(test_query, connection)
# Display the DataFrame
print(df)
# Close Connection if necessary
connection.close()

## 2. Data Exploration

[Click here](https://www.nba.com/stats/help/glossary) for an NBA stat glossary.
[Click here](https://en.wikipedia.org/wiki/Basketball_statistics) for general basketball statistics acronyms.

Slug for CSV folder files (for the purpose of viewing the tables here): /basketball/csv/[table name].csv

* fgm = field goals made
* ftm = free throws made
* pts = points
* reb = rebounds
* oreb = offensive rebounds
* dreb = defensive rebounds
* ast = assists
* stl = steals
* blk = blocks
* tov = turnovers
* td = triple double
* wl = win/lose
* ft_pct_away = free throws % of away team

NBA's efficiency rating: (PTS + REB + AST + STL + BLK − ((FGA − FGM) + (FTA − FTM) + TOV))

## 3. Data Cleaning

In [ ]:
# 1.Connect to SQL database
explore_connection = sql.connect('basketball/nba.sqlite')

cursor = explore_connection.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS game_cleaned AS
    SELECT team_id_home, 
        team_id_away, 
        team_name_home, 
        team_name_away, 
        game_id, 
        wl_home, 
        fga_home, 
        fg_pct_home, 
        ftm_home, 
        pts_home, 
        fta_home, 
        fgm_home, 
        ftm_away, 
        oreb_home, 
        dreb_home, 
        reb_home, 
        ast_home, 
        stl_home, 
        blk_home, 
        tov_home, 
        pf_home, 
        team_id_away, 
        team_name_away, 
        wl_away, 
        fg_pct_away, 
        ft_pct_away, 
        fta_away, 
        pts_away, 
        fga_away, 
        fgm_away, 
        oreb_away, 
        dreb_away, 
        reb_away, 
        ast_away, 
        stl_away, 
        blk_away, 
        tov_away, 
        pf_away
    FROM game
    WHERE fga_home IS NOT NULL AND fta_home IS NOT NULL AND ftm_home IS NOT NULL AND oreb_home IS NOT NULL AND dreb_home IS NOT NULL AND stl_home IS NOT NULL AND ast_home IS NOT NULL AND reb_home IS NOT NULL AND blk_home IS NOT NULL AND tov_home IS NOT NULL AND pf_home IS NOT NULL AND pts_home IS NOT NULL AND team_id_away IS NOT NULL AND team_name_away IS NOT NULL AND wl_away IS NOT NULL AND fg_pct_away IS NOT NULL AND ft_pct_away IS NOT NULL AND pts_away IS NOT NULL AND fgm_away IS NOT NULL AND oreb_away IS NOT NULL AND dreb_away IS NOT NULL AND reb_away IS NOT NULL AND ast_away IS NOT NULL AND stl_away IS NOT NULL AND blk_away IS NOT NULL AND tov_away IS NOT NULL AND pf_away IS NOT NULL AND pts_away IS NOT NULL AND fga_away IS NOT NULL;
''')
display_game_cleaned_query = "SELECT * FROM game_cleaned"
game_cleaned_df = pd.read_sql_query(display_game_cleaned_query, explore_connection)
print(game_cleaned_df)

<h4>I noticed that in the teams.csv the Atlanta Hawks have the wrong state in the table (It says "Atlanta" again for the state). Here, I clean this by replacing that value with "Georgia".</h4>

In [ ]:
# 1.Connect to SQL database
fixing_state_value_connection = sql.connect('basketball/nba.sqlite')
cursor = fixing_state_value_connection.cursor()
cursor.execute('''
    UPDATE team
    SET state = 'Georgia'
    WHERE id = 1610612737
''')


display_team_fixed_query = "SELECT * FROM team"
team_cleaned_df = pd.read_sql_query(display_team_fixed_query, fixing_state_value_connection)
print(team_cleaned_df)
fixing_state_value_connection.close()

## 3. Data Wrangling

### Calculating the efficiency of the home vs away teams for each game

In [ ]:
# 1.Connect to SQL database
add_calculation_connection = sql.connect('basketball/nba.sqlite')
cursor = add_calculation_connection.cursor()
cursor.execute('''
    ALTER TABLE game_cleaned ADD COLUMN eff_home;
''')

cursor.execute('''
    ALTER TABLE game_cleaned ADD COLUMN eff_away;
''')

cursor.execute('''
    UPDATE game_cleaned
    SET eff_home = (
        pts_home + 
        reb_home + 
        ast_home + 
        stl_home + 
        blk_home - 
        ((fga_home - fgm_home) + 
        (fta_home - ftm_home) + 
        tov_home)
    ),
    eff_away = (
        pts_away + 
        reb_away + 
        ast_away + 
        stl_away + 
        blk_away - 
        ((fga_away - fgm_away) + 
        (fta_away - ftm_away) + 
        tov_away)
    )
''')

display_game_calculated_query = "SELECT * FROM game_cleaned"
game_calculated_df = pd.read_sql_query(display_game_calculated_query, add_calculation_connection)
print(game_calculated_df)
add_calculation_connection.close()

### Duplicate the team table so that I can have a city for each team in the game_cleaned table

In [ ]:
duplicate_team_table_connection = sql.connect('basketball/nba.sqlite')
cursor = duplicate_team_table_connection.cursor()
cursor.execute('''
    CREATE TABLE duplicate_team_table AS
    SELECT * FROM team
''')
duplicate_team_table_connection.closed()

### Joining the duplicated table with the cleaned game table, rename the city field AND state field for the home team

In [ ]:
join_duplicated_team_w_cleaned_game = sql.connect('basketball/nba.sqlite')
cursor = join_duplicated_team_w_cleaned_game.cursor()
cursor.execute('''
      SELECT 
        game_cleaned.team_id_home, 
        game_cleaned.team_name_home, 
        game_cleaned.team_name_away, 
        game_cleaned.game_id,
        game_cleaned.eff_home,
        game_cleaned.eff_away,
        duplicate_team_table.state AS state_away,
        duplicate_team_table.city AS city_away
    FROM game_cleaned
    INNER JOIN duplicate_team_table ON game_cleaned.team_id_away = duplicate_team_table.id;
''')
display_game_team_duplicate_joined_query = "SELECT * FROM game_cleaned"
game_team_duplicated_df = pd.read_sql_query(display_game_team_duplicate_joined_query, join_duplicated_team_w_cleaned_game)
print(game_team_duplicated_df)
join_duplicated_team_w_cleaned_game.closed()

### Joining the original team Table with the cleaned Game table and renaming the city field AND state field for the away team

In [ ]:
join_teams_original_and_games_cleaned_connection = sql.connect('basketball/nba.sqlite')
cursor = join_teams_original_and_games_cleaned_connection.cursor()
cursor.execute('''
    SELECT  
        game_cleaned.team_id_away, 
        game_cleaned.team_name_home, 
        game_cleaned.team_name_away, 
        game_cleaned.game_id,
        game_cleaned.eff_home,
        game_cleaned.eff_away,
        game_cleaned.state_away,
        game_cleaned.city_away,
        team.state AS state_home,
        team.city AS state_away
    FROM game_cleaned
    INNER JOIN team ON game_cleaned.team_id_home = team.id;
''')


display_fully_joined_game_query = "SELECT * FROM game_cleaned"
game_fully_joined_df = pd.read_sql_query(display_fully_joined_game_query, join_teams_original_and_games_cleaned_connection)
print(game_fully_joined_df)
join_teams_original_and_games_cleaned_connection.close()

### Add a column with the distance between the two cities COORDINATES for each game.


#### Calculate timezone, longitude and latitude

In [ ]:
# Find timezone, longitude and latitude, and distance between the cities. Add those values to the table

timezone_and_distance_connection = sql.connect('basketball/nba.sqlite')
cursor = timezone_and_distance_connection.cursor()
cursor.execute('''
    ALTER TABLE game_cleaned ADD COLUMN latitude_home REAL;
    ALTER TABLE game_cleaned ADD COLUMN longitude_home REAL;
    ALTER TABLE game_cleaned ADD COLUMN timezone_home TEXT;
    ALTER TABLE game_cleaned ADD COLUMN latitude_away REAL;
    ALTER TABLE game_cleaned ADD COLUMN longitude_away REAL;
    ALTER TABLE game_cleaned ADD COLUMN timezone_away TEXT;
    ALTER TABLE game_cleaned ADD COLUMN distance_away_from_home REAL;
''')

query = "SELECT city FROM game_cleaned"
query_away = "SELECT city_away FROM game_cleaned"

geolocator = Nominatim(user_agent="geoapi")
tf = TimezoneFinder()
'''
Find the latitude, longitude, timezone of each city. Find the distance from home for the away team.
'''
def get_location_data(city):
    try:
        # Get city location
        location = geolocator.geocode(city)
        if location:
            lat, lon = location.latitude, location.longitude
            # Get timezone based on latitude and longitude
            timezone = tf.timezone_at(lat=lat, lng=lon)
            return lat, lon, timezone
        else:
            return None, None, None
    except Exception as e:
        print(f"Error with city {city}: {e}")
        return None, None, None

# Apply the function to each city and store the results in new columns
df['latitude_home'], df['longitude_home'], df['timezone_home'] = zip(*df['city'].apply(get_location_data))
df['latitude_away'], df['longitude_away'], df['timezone_away'] = zip(*df['city_away'].apply(get_location_data))

# Iterate over the DataFrame and update the table for the HOME team
for index, row in df.iterrows():
    city = row['city']
    lat = row['latitude_home']
    lon = row['longitude_home']
    timezone = row['timezone_home']
    
    # Update the table with the latitude, longitude, and timezone for each city
    cursor.execute("""
        UPDATE game_cleaned 
        SET latitude_home = ?, longitude_home = ?, timezone_home = ? 
        WHERE team_city = ?""", (lat, lon, timezone, city))

# Iterate over the DataFrame and update the table for the AWAY team

for index, row in df.iterrows():
    city = row['city_away']
    lat = row['latitude_away']
    lon = row['longitude_away']
    timezone = row['timezone_away']
    
    # Update the table with the latitude, longitude, and timezone for each city
    cursor.execute("""
        UPDATE game_cleaned 
        SET latitude_away = ?, longitude_away = ?, timezone_away = ? 
        WHERE team_city_away = ?""", (lat, lon, timezone, city))

timezone_and_distance_connection.commit()

display_timezones_dist_query = "SELECT * FROM game_cleaned"
game_timezones_df = pd.read_sql_query(display_timezones_dist_query, timezone_and_distance_connection)
print(game_timezones_df)
timezone_and_distance_connection.close()

#### Calculate distance the away team traveled with the coordinates